![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# Classificação de Atividade Humana com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier

from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import cross_val_score, train_test_split

filename_features = "Dados/UCI HAR Dataset/features.txt"
filename_labels = "Dados/UCI HAR Dataset/activity_labels.txt"

filename_subtrain = "Dados/UCI HAR Dataset/train/subject_train.txt"
filename_xtrain = "Dados/UCI HAR Dataset/train/X_train.txt"
filename_ytrain = "Dados/UCI HAR Dataset/train/y_train.txt"

filename_subtest = "Dados/UCI HAR Dataset/test/subject_test.txt"
ffilename_xtest = "Dados/UCI HAR Dataset/test/X_test.txt"
filename_ytest = "Dados/UCI HAR Dataset/test/y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#")
features = features.squeeze('columns')
labels = pd.read_csv(filename_labels, sep='\s+', header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])
subject_train = subject_train.squeeze('columns')
X_train = pd.read_csv(filename_xtrain, sep='\s+', header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])
subject_test = subject_test.squeeze('columns')
X_test = pd.read_csv(ffilename_xtest, sep='\s+', header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

## PCA com variáveis padronizadas

Reflexão sobre a escala das variáveis:

**Variáveis em métricas muito diferentes** podem interferir na análise de componentes principais. Lembra que variância é informação pra nós? Pois bem, tipicamente se há uma variável monetária como salário, vai ter uma ordem de variabilidade bem maior que número de filhos, tempo de emprego ou qualquer variável dummy. Assim, as variáveis de maior variância tendem a "dominar" a análise. Nesses casos é comum usar a padronização das variáveis.

Faça duas análises de componentes principais para a base do HAR - com e sem padronização e compare:

- A variância explicada por componente
- A variância explicada acumulada por componente
- A variância percentual por componente
- A variância percentual acumulada por componente
- Quantas componentes você escolheria, em cada caso para explicar 90% da variância?

In [8]:
%%time

def padroniza(s):
    if s.std() > 0:
        s = (s - s.mean())/s.std()
    return s

X_train_pad = pd.DataFrame(X_train).apply(padroniza, axis=0)
X_test_pad = pd.DataFrame(X_test).apply(padroniza, axis=0)
X_train_pad.head()

CPU times: total: 469 ms
Wall time: 998 ms


,1 tBodyAcc-mean()-X,2 tBodyAcc-mean()-Y,3 tBodyAcc-mean()-Z,4 tBodyAcc-std()-X,5 tBodyAcc-std()-Y,6 tBodyAcc-std()-Z,7 tBodyAcc-mad()-X,8 tBodyAcc-mad()-Y,9 tBodyAcc-mad()-Z,10 tBodyAcc-max()-X,...,552 fBodyBodyGyroJerkMag-meanFreq(),553 fBodyBodyGyroJerkMag-skewness(),554 fBodyBodyGyroJerkMag-kurtosis(),"555 angle(tBodyAccMean,gravity)","556 angle(tBodyAccJerkMean),gravityMean)","557 angle(tBodyGyroMean,gravityMean)","558 angle(tBodyGyroJerkMean,gravityMean)","559 angle(X,gravityMean)","560 angle(Y,gravityMean)","561 angle(Z,gravityMean)"
0,0.200628,-0.063678,-0.419600,-0.868755,-0.939377,-0.737479,-0.859758,-0.938955,-0.766385,-0.855978,...,-0.795305,0.025958,-0.276380,-0.360579,0.062935,-0.778374,-0.026079,-0.687172,0.407918,-0.007567
1,0.055944,0.031484,-0.253891,-0.875366,-0.923839,-0.849247,-0.868472,-0.921936,-0.848870,-0.871300,...,0.130605,-0.897296,-0.767938,0.133002,-0.021460,-1.218722,1.484369,-0.694091,0.409089,0.007875
2,0.073510,-0.043414,-0.076289,-0.868980,-0.907698,-0.893724,-0.863078,-0.898793,-0.896640,-0.863264,...,1.152257,-0.260860,-0.438286,-0.377815,0.391949,0.151197,1.704085,-0.702191,0.410260,0.026501
3,0.066691,-0.208407,-0.249695,-0.870566,-0.939959,-0.921743,-0.864445,-0.938060,-0.925216,-0.863264,...,1.112694,0.591005,0.463123,-0.135016,-0.033635,1.037781,-1.002951,-0.701636,0.414622,0.031712
4,0.030467,0.027585,-0.109840,-0.875128,-0.934815,-0.921281,-0.867325,-0.931726,-0.927965,-0.870201,...,-0.149567,-0.138505,-0.240296,0.340383,0.268468,1.125841,-1.276196,-0.700104,0.425434,0.045222


## Árvore com PCA

Faça duas uma árvore de decisão com 10 componentes principais - uma com base em dados padronizados e outra sem padronizar. Utilize o ```ccp_alpha=0.001```.

Compare a acurácia na base de treino e teste.

##### Função para avaliação da acurácia das árvores treinadas:

In [7]:
def avalia (model: pd.DataFrame, conjunto_X: list, conjunto_y:list, nomes=['treino', 'teste'], exibir = True):
    resultados = {}
    for nome, X, y in zip(nomes, conjunto_X, conjunto_y):
        scores = model.score(X, y)
        resultados[nome]= scores
        if exibir:
            print(f'Acurácia na base de {nome}: {scores*100:.1f}' )
    return resultados

##### Seleção da 10 componentes principais na base padronizada:

In [9]:
%%time

n=10

prcomp_pad = PCA().fit(X_train_pad)

pc_treino_pad = prcomp.transform(X_train_pad)
pc_teste_pad  = prcomp.transform(X_test_pad)

colunas = ['cp'+str(x+1) for x in list(range(n))]

pc_train_pad = pd.DataFrame(pc_treino_pad[:,:n], columns = colunas)
pc_test_pad  = pd.DataFrame(pc_teste_pad[:,:n], columns = colunas)

pc_train_pad.head()

CPU times: total: 891 ms
Wall time: 1.27 s


,cp1,cp2,cp3,cp4,cp5,cp6,cp7,cp8,cp9,cp10
0,-5.056926,3.248497,-0.705398,7.114440,5.869270,1.235046,1.970115,2.646722,-4.351006,-0.916955
1,-4.622151,3.342921,-2.055620,4.669840,2.377596,2.382478,-1.024350,0.011686,-2.111754,-1.889907
2,-4.464275,4.488619,-2.702197,4.114478,1.290369,1.403118,0.113257,1.769423,-0.886643,-0.992741
3,-4.845874,6.239507,-2.255582,3.199217,2.364289,3.200693,-0.200014,0.432466,-0.632696,0.478751
4,-5.062704,7.056044,-2.503758,3.096679,1.569520,2.329194,-0.525606,1.527188,-0.310758,0.575354


##### Treinamento e avaliação da acurácia na base padronizada:

In [23]:
clf_pad = DecisionTreeClassifier(ccp_alpha=0.001, random_state=23).fit(pc_train_pad, y_train)

resultado_pad = avalia(clf_pad,[pc_train_pad, pc_test_pad], [y_train, y_test])

Acurácia na base de treino: 89.2
Acurácia na base de teste: 83.2


##### Seleção da 10 componentes principais na base original:

In [13]:
%%time

n=10

prcomp = PCA().fit(X_train)

pc_treino = prcomp.transform(X_train)
pc_teste  = prcomp.transform(X_test)

colunas = ['cp'+str(x+1) for x in list(range(n))]

pc_train = pd.DataFrame(pc_treino[:,:n], columns = colunas)
pc_test  = pd.DataFrame(pc_teste[:,:n], columns = colunas)

pc_train.head()

CPU times: total: 1.11 s
Wall time: 1.21 s


,cp1,cp2,cp3,cp4,cp5,cp6,cp7,cp8,cp9,cp10
0,-5.520280,-0.290278,-1.529929,1.333242,1.425089,-0.194708,0.577454,0.691490,-1.222865,-0.363414
1,-5.535350,-0.082530,-1.924804,0.671273,0.671260,0.735144,-0.616908,-0.771714,-0.615496,-0.895525
2,-5.474988,0.287387,-2.144642,0.531806,0.207824,-0.037772,0.057628,0.093917,-0.063147,-0.216898
3,-5.677232,0.897031,-2.018220,0.157125,0.759085,1.079547,-0.267805,-0.731391,0.281296,0.466269
4,-5.748749,1.162952,-2.139533,0.207823,0.473090,0.463035,-0.152227,-0.107509,0.289819,0.539206


##### Treinamento e avaliação da acurácia na base original:

In [24]:
clf = DecisionTreeClassifier(ccp_alpha=0.001, random_state=23).fit(pc_train, y_train)

resultado = avalia(clf,[pc_train, pc_test], [y_train, y_test])

Acurácia na base de treino: 89.3
Acurácia na base de teste: 82.4


##### Comparativo do valor da acurácia entre as bases original e padronizada:

In [35]:
resultado_acc = {}

resultado_acc['original'] = resultado
resultado_acc['padronizado'] = resultado_pad

pd.DataFrame(resultado_acc).T

,treino,teste
original,0.892682,0.823889
padronizado,0.891866,0.832033


O valor da acurácia na base original apresenta uma amplitude maior entre treino e teste do que a base padronizada. No qual, para a base padronizada em relação a base original o valor da acurácia diminui levemente no treino e aumenta levemente no teste.